In [2]:
%serialconnect to --port=COM3 --baud=115200

Connecting to --port=COM3 --baud=115200 
OSError: [Errno 2] ENOENT
MicroPython v1.10-298-g47e76b527 on 2019-04-18; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [3]:
from machine import Pin 
import sys
import ustruct as struct
import utime
from machine import Pin, SPI
from nrf24l01 import NRF24L01
from nrf24l01 import *
from micropython import const
import time

In [4]:
# nRF24L01+ registers
CONFIG      = const(0x00)
EN_RXADDR   = const(0x02)
SETUP_AW    = const(0x03)
SETUP_RETR  = const(0x04)
RF_CH       = const(0x05)
RF_SETUP    = const(0x06)
STATUS      = const(0x07)
RX_ADDR_P0  = const(0x0a)
TX_ADDR     = const(0x10)
RX_PW_P0    = const(0x11)
FIFO_STATUS = const(0x17)
DYNPD = const(0x1c)

# CONFIG register
EN_CRC      = const(0x08) # enable CRC
CRCO        = const(0x04) # CRC encoding scheme; 0=1 byte, 1=2 bytes
PWR_UP      = const(0x02) # 1=power up, 0=power down
PRIM_RX     = const(0x01) # RX/TX control; 0=PTX, 1=PRX

# RF_SETUP register
POWER_0     = const(0x00) # -18 dBm
POWER_1     = const(0x02) # -12 dBm
POWER_2     = const(0x04) # -6 dBm
POWER_3     = const(0x06) # 0 dBm
SPEED_1M    = const(0x00)
SPEED_2M    = const(0x08)
SPEED_250K  = const(0x20)
CONT_WAVE = 0x80
PLL_LOCK = 0x10

# STATUS register
RX_DR       = const(0x40) # RX data ready; write 1 to clear
TX_DS       = const(0x20) # TX data sent; write 1 to clear
MAX_RT      = const(0x10) # max retransmits reached; write 1 to clear

# FIFO_STATUS register
RX_EMPTY    = const(0x01) # 1 if RX FIFO is empty

# constants for instructions
R_RX_PL_WID  = const(0x60) # read RX payload width
R_RX_PAYLOAD = const(0x61) # read RX payload
W_TX_PAYLOAD = const(0xa0) # write TX payload
FLUSH_TX     = const(0xe1) # flush TX FIFO
FLUSH_RX     = const(0xe2) # flush RX FIFO
NOP          = const(0xff) # use to read STATUS register

In [5]:
cfg = {'spi': -1, 'miso': 32, 'mosi': 33, 'sck': 25, 'csn': 26, 'ce': 27}

print('NRF24L01 pinout for test:') 
print('    CE on', cfg['ce']) 
print('    CSN on', cfg['csn']) 
print('    SCK on', cfg['sck']) 
print('    MISO on', cfg['miso']) 
print('    MOSI on', cfg['mosi']) 

csn = Pin(cfg['csn'], mode=Pin.OUT, value=1) 
ce = Pin(cfg['ce'], mode=Pin.OUT, value=0) 
spi = SPI(-1, sck=Pin(cfg['sck']), mosi=Pin(cfg['mosi']), miso=Pin(cfg['miso'])) 

buf = bytearray(1)

def print_bin(val):
    print("{0:{fill}8b}".format(val, fill=0))

def reg_read(reg):
    csn(0)
    spi.readinto(buf, reg)
    spi.readinto(buf)
    csn(1)
    return buf[0]

def reg_write(reg, value):
    csn(0)
    spi.readinto(buf, 0x20 | reg)
    ret = buf[0]
    spi.readinto(buf, value)
    csn(1)
    return ret    
    


NRF24L01 pinout for test:
    CE on 27
    CSN on 26
    SCK on 25
    MISO on 32
    MOSI on 33


In [6]:
#Initialize SPI


try:
    master = spi.MASTER
except AttributeError:
    spi.init(baudrate=4000000, polarity=0, phase=0)
else:
    spi.init(master, baudrate=4000000, polarity=0, phase=0)
# reset everything
ce.init(ce.OUT, value=0)
csn.init(csn.OUT, value=1)


#nrf = NRF24L01(spi, csn, ce, payload_size=8, channel=1) 


In [7]:
config = 0
reg_write(CONFIG, config)
config = reg_read(CONFIG)
print_bin(config)
config |= PWR_UP
config &= ~PRIM_RX 
reg_write(CONFIG, config)
config = reg_read(CONFIG)
print_bin(config)
utime.sleep_ms(10)

00000000
00000010


In [8]:
rfreg = 0
rfreg = reg_write(RF_SETUP, rfreg)
rfreg = reg_read(RF_SETUP)
print_bin(rfreg)
rfreg |= CONT_WAVE
rfreg |= PLL_LOCK
rfreg |= POWER_0
print_bin(rfreg)
rfreg = reg_write(RF_SETUP, rfreg)
rfreg = reg_read(RF_SETUP)
print_bin(rfreg)
reg_write(RF_CH, 2)
ce(1)

00000000
10010000
10010000


In [9]:
ce(1)

In [16]:
for i in range(1000):
    ce(1)
    utime.sleep_ms(100)
    ce(0)
    utime.sleep_ms(100)

..................................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 5, in <module>
KeyboardInterrupt: 
